# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import ow_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"City count: {len(cities)}")

City count: 586


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
## Set OpenWeather API base url
ow_url = "http://api.openweathermap.org/data/2.5/weather?q="
## Set Units
units = "imperial"

## Create weather data empty list
weather_data = []
## Set initial values
record = 1

## Print Log Header
print("Beginning Data Retrieval")
print("------------------------------")

for city in cities:
        ##
        query_url = f"{ow_url}{city}&appid={ow_key}&units={units}"
        response = requests.get(query_url).json()
        try:
            ## Print Log Status
            print(f"Processing Record {record:>2d} | City Id = {response['id']:<7d} [{city}]")
            ## Retrieve weather data from OpenWeather API (response)
            weather_data.append({
                'City': response['name'],
                'Lat': response['coord']['lat'],
                'Lng': response['coord']['lon'],
                'Max Temp': response['main']['temp_max'],
                'Humidity': response['main']['humidity'],
                'Cloudiness': response['clouds']['all'],
                'Wind Speed': response['wind']['speed'],
                'Country': response['sys']['country'],
                'Date': response['dt']
            })
            ## Record counter
            record += 1
        except KeyError:
            ## Print Log Status
            print(f"City [{city}] not found. Skipping...")
            pass

## Print Log Footer
print("------------------------------")        
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
------------------------------
Processing Record  1 | City Id = 2278158 [buchanan]
City [illoqqortoormiut] not found. Skipping...
Processing Record  2 | City Id = 3622323 [pocosol]
Processing Record  3 | City Id = 2070998 [geraldton]
Processing Record  4 | City Id = 2126682 [bilibino]
Processing Record  5 | City Id = 2538474 [rabat]
Processing Record  6 | City Id = 6201424 [mataura]
Processing Record  7 | City Id = 4031574 [provideniya]
Processing Record  8 | City Id = 1505074 [kalachinsk]
Processing Record  9 | City Id = 3471451 [arraial do cabo]
Processing Record 10 | City Id = 2022572 [khatanga]
Processing Record 11 | City Id = 4030556 [rikitea]
City [tsihombe] not found. Skipping...
Processing Record 12 | City Id = 1507390 [dikson]
Processing Record 13 | City Id = 2168305 [dubbo]
Processing Record 14 | City Id = 1624041 [ternate]
Processing Record 15 | City Id = 3377301 [mabaruma]
Processing Record 16 | City Id = 110690  [faya]
Processing Record 17 | City I

Processing Record 149 | City Id = 934322  [mahebourg]
Processing Record 150 | City Id = 1267390 [kavaratti]
Processing Record 151 | City Id = 3443061 [chuy]
Processing Record 152 | City Id = 3386213 [touros]
Processing Record 153 | City Id = 2077895 [alice springs]
Processing Record 154 | City Id = 232066  [hima]
Processing Record 155 | City Id = 777019  [vardo]
Processing Record 156 | City Id = 2514301 [mahon]
City [nizhneyansk] not found. Skipping...
Processing Record 157 | City Id = 2122783 [katangli]
Processing Record 158 | City Id = 2126199 [cherskiy]
Processing Record 159 | City Id = 3423146 [ilulissat]
Processing Record 160 | City Id = 777682  [skjervoy]
Processing Record 161 | City Id = 2082539 [merauke]
Processing Record 162 | City Id = 4035715 [avarua]
Processing Record 163 | City Id = 3625346 [villa bruzual]
Processing Record 164 | City Id = 1045114 [inhambane]
Processing Record 165 | City Id = 2294877 [tamale]
City [umzimvubu] not found. Skipping...
Processing Record 166 | 

City [samalaeulu] not found. Skipping...
Processing Record 297 | City Id = 3698359 [chicama]
Processing Record 298 | City Id = 286245  [sur]
Processing Record 299 | City Id = 2178753 [kirakira]
Processing Record 300 | City Id = 3379507 [bartica]
Processing Record 301 | City Id = 2036069 [linjiang]
Processing Record 302 | City Id = 4993659 [garden city]
Processing Record 303 | City Id = 1274868 [chalisgaon]
Processing Record 304 | City Id = 2121385 [severo-kurilsk]
Processing Record 305 | City Id = 2527087 [tiznit]
Processing Record 306 | City Id = 5921357 [chilliwack]
Processing Record 307 | City Id = 373141  [kaduqli]
Processing Record 308 | City Id = 2450173 [taoudenni]
Processing Record 309 | City Id = 5808079 [redmond]
Processing Record 310 | City Id = 2381334 [atar]
Processing Record 311 | City Id = 2152659 [port macquarie]
Processing Record 312 | City Id = 2034599 [tailai]
Processing Record 313 | City Id = 3899695 [ancud]
Processing Record 314 | City Id = 2122262 [palana]
Process

Processing Record 444 | City Id = 1789065 [ankang]
Processing Record 445 | City Id = 3203982 [bjelovar]
Processing Record 446 | City Id = 1627185 [sijunjung]
Processing Record 447 | City Id = 1489656 [teya]
Processing Record 448 | City Id = 145531  [saurimo]
Processing Record 449 | City Id = 3394023 [natal]
Processing Record 450 | City Id = 2138555 [poum]
Processing Record 451 | City Id = 1261853 [narasannapeta]
Processing Record 452 | City Id = 3391395 [porto da folha]
Processing Record 453 | City Id = 1500764 [lebyazhye]
Processing Record 454 | City Id = 3405061 [boqueirao]
Processing Record 455 | City Id = 287832  [ibra]
City [urdzhar] not found. Skipping...
Processing Record 456 | City Id = 2155562 [nelson bay]
Processing Record 457 | City Id = 3470137 [belmonte]
Processing Record 458 | City Id = 1645528 [denpasar]
Processing Record 459 | City Id = 295277  [elat]
Processing Record 460 | City Id = 3115824 [muros]
City [louisbourg] not found. Skipping...
Processing Record 461 | City 

In [4]:
# Print the city count to confirm updated count
print(f"Updated city count: {len(weather_data)}")

Updated city count: 545


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
pd.DataFrame(data=weather_data)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Buchanan,5.8808,-10.0467,89.60,74,75,6.91,LR,1619537484
1,Pocosol,10.3667,-84.6167,78.80,57,40,11.50,CR,1619537484
2,Geraldton,-28.7667,114.6000,62.60,88,4,3.44,AU,1619537485
3,Bilibino,68.0546,166.4372,3.81,98,90,2.75,RU,1619537485
4,Rabat,33.9911,-6.8401,68.00,73,40,10.36,MA,1619537483
...,...,...,...,...,...,...,...,...,...
540,Medicine Hat,50.0501,-110.6683,50.00,49,1,16.11,CA,1619537665
541,Lagoa,39.0500,-27.9833,60.08,76,93,10.27,PT,1619537665
542,Bagdarin,54.4333,113.6000,21.47,52,100,9.28,RU,1619537666
543,Oistins,13.0667,-59.5333,84.20,54,20,19.57,BB,1619537389


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
#  Get the indices of cities that have humidity over 100%.


In [7]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression